In [1]:
import pandas as pd
import json
import gzip
import os

file = "/home/marc/inters/output_test2/05_Annotation/Nirvana/snvs/Annotation.json.gz"
gene_info = "/home/marc/Desktop/data/refs/gtf/GRCH38_91/gene.info"
Jasix = os.path.expanduser("$HOME/Nirvana/bin/Release/net6.0/Jasix.dll")

In [42]:
id = "FAM138A"
padding = 1000

In [58]:
def extract_gene_region(Ann_file, gene_info, id, padding=1000):
    interval = os.popen(f"grep -i 'Name={id};' {gene_info} | awk -F'\t' '{{print \"chr\"$1\":\"$4-{padding}\"-\"$5+{padding}}}'").read().strip()
    if interval == '':
        print("didn't find Gene in Gene list!")
        return None
    else:
        print(F"Exporting {id} with padding [ {padding} ] ({interval})  from Annotation file...")
        dotnet_command = f"dotnet {Jasix} -i {file} -q {interval}"
        dotnet_output = os.popen(dotnet_command)
        dotnet_output_str = dotnet_output.read().replace("\n","")
        dotnet_output_exitcode = dotnet_output.close()
        if dotnet_output_exitcode is not None:
            print(f"ERROR with dotnet command! maybe didn't find the gene {id}")
            return None
        else:
            formatted_output = dotnet_output_str.replace("{  \"chromosome", "\n{\"chromosome")
            formatted_output = formatted_output.replace(" ", "")
            formatted_output = formatted_output.replace('{"positions":[\n{', '[{').replace("}]}]}]}","}]}]}]")

            return pd.DataFrame(json.loads(formatted_output))

def process_region(*args):
    counter = 0
    for df in args:
        if counter == 0:
            my_df = df
        else:
            my_df = pd.concat([my_df, df], axis=0, ignore_index=True)
        counter += 1
        
    return my_df

def process_dataframe(df):
    df = pd.DataFrame(df)
    try:
        variants_df = df['variants'].apply(pd.Series)
        variants_df2 = variants_df[0].apply(pd.Series)
        final_df = pd.concat([df, variants_df2], axis=1).drop(['variants', 'samples'], axis=1)
        return final_df

    except:
        print("No match found for any of the provided genes!")



def print_stats(final_df):
    try:
        total_rows = len(final_df)
        total_variants = total_rows
        variants_with_annotations = total_rows - final_df['dbsnp'].isna().sum()
        variants_with_regulatoryRegions = total_rows - final_df['regulatoryRegions'].isna().sum()
        variants_with_inlowcomplexity = total_rows - final_df['inLowComplexityRegion'].isna().sum()
        novel_variants = total_variants - variants_with_annotations
        percentage_novel = round(((novel_variants / total_rows) * 100),2)
        print(f"Total Variants: {total_variants}")
        print(f"No. Variants with dbSNP Annotations: {variants_with_annotations}")
        print(f"No. Variants affecting regualtory regions: {variants_with_regulatoryRegions}")
        print(f"No. Variants in low complexity regions: {variants_with_inlowcomplexity}")
        print(f"Percentage of Unknown Variants (dbsnp only): {percentage_novel} %")
    except:
        exit(1)

In [59]:
egene = extract_gene_region(file, gene_info, "OR4F21", 5000)
egene2 = extract_gene_region(file, gene_info, "FBXO16", 5000)
egene3 = extract_gene_region(file, gene_info, "FGL1", 5000)
egene4 = extract_gene_region(file, gene_info, "c", 5000)


Exporting OR4F21 with padding [ 5000 ] (chr8:161049-172043)  from Annotation file...
Exporting FBXO16 with padding [ 5000 ] (chr8:28343287-28495318)  from Annotation file...
Exporting FGL1 with padding [ 5000 ] (chr8:17859380-17915365)  from Annotation file...
Exporting FAM138A with padding [ 5000 ] (chr1:29554-41081)  from Annotation file...
ERROR with dotnet command! maybe didn't find the gene FAM138A


In [60]:
combined = process_region(egene4)
final_df = print_stats(process_dataframe(combined))

No match found for any of the provided genes!


In [61]:
combined = process_region(egene,egene2,egene3,egene4)
final_df = print_stats(process_dataframe(combined))

Total Variants: 54
No. Variants with dbSNP Annotations: 5
No. Variants affecting regualtory regions: 5
No. Variants in low complexity regions: 3
Percentage of Unknown Variants (dbsnp only): 90.74 %


In [50]:
final_df

,chromosome,position,refAllele,altAlleles,quality,filters,fisherStrandBias,mappingQuality,cytogeneticBand,vid,...,hgvsg,phylopScore,dannScore,transcripts,dbsnp,gnomad,topmed,gerpScore,regulatoryRegions,inLowComplexityRegion
0,chr8,161062,C,[T],3448.97,[PASS],12.287,60.00,8p23.3,8-161062-C-T,...,NC_000008.11:g.161062C>T,0.2,0.36,"[{'transcript': 'ENST00000320901.3', 'source':...",NaN,NaN,NaN,NaN,NaN,NaN
1,chr8,161167,C,[T],585.97,[PASS],0.000,60.00,8p23.3,8-161167-C-T,...,NC_000008.11:g.161167C>T,-1.2,0.64,"[{'transcript': 'ENST00000320901.3', 'source':...",[rs200059412],NaN,NaN,NaN,NaN,NaN
2,chr8,161176,T,[C],598.97,[PASS],0.000,60.00,8p23.3,8-161176-T-C,...,NC_000008.11:g.161176T>C,0.2,0.33,"[{'transcript': 'ENST00000320901.3', 'source':...",[rs1241218637],"{'coverage': 0, 'failedFilter': True, 'allAf':...","{'allAf': 8e-06, 'allAn': 125568, 'allAc': 1, ...",NaN,NaN,NaN
3,chr8,161240,A,[G],218.29,[PASS],0.000,60.00,8p23.3,8-161240-A-G,...,NC_000008.11:g.161240A>G,0.2,NaN,"[{'transcript': 'ENST00000320901.3', 'source':...",NaN,NaN,NaN,-0.7420,NaN,NaN
4,chr8,163226,T,[C],8887.16,[PASS],5.194,60.00,8p23.3,8-163226-T-C,...,NC_000008.11:g.163226T>C,0.2,NaN,"[{'transcript': 'ENST00000320901.3', 'source':...",NaN,NaN,NaN,NaN,NaN,NaN
5,chr8,163249,T,[C],8929.16,[PASS],9.989,60.00,8p23.3,8-163249-T-C,...,NC_000008.11:g.163249T>C,0.2,0.44,"[{'transcript': 'ENST00000320901.3', 'source':...",NaN,NaN,NaN,NaN,NaN,NaN
6,chr8,163366,T,[C],16714.98,[PASS],0.000,59.98,8p23.3,8-163366-T-C,...,NC_000008.11:g.163366T>C,0.2,0.47,"[{'transcript': 'ENST00000320901.3', 'source':...",[rs1376642346],"{'coverage': 0, 'allAf': 1.4e-05, 'allAn': 148...","{'allAf': 1.6e-05, 'allAn': 125568, 'allAc': 2...",1.2900,NaN,NaN
7,chr8,163370,C,[G],43882.73,[PASS],0.000,59.98,8p23.3,8-163370-C-G,...,NC_000008.11:g.163370C>G,0.2,0.39,"[{'transcript': 'ENST00000320901.3', 'source':...",NaN,NaN,NaN,2.7400,NaN,NaN
8,chr8,163387,C,[T],11856.16,[PASS],9.653,59.99,8p23.3,8-163387-C-T,...,NC_000008.11:g.163387C>T,0.2,NaN,"[{'transcript': 'ENST00000320901.3', 'source':...",NaN,NaN,NaN,NaN,NaN,NaN
9,chr8,163432,A,[G],12922.98,[PASS],22.033,59.96,8p23.3,8-163432-A-G,...,NC_000008.11:g.163432A>G,0.2,0.67,"[{'transcript': 'ENST00000320901.3', 'source':...",NaN,NaN,NaN,1.9900,NaN,NaN


In [6]:
final_df.to_csv("/home/marc/example.tsv", sep="\t", index = False)

In [8]:
for i in range(0,5):
    print(final_df['regulatoryRegions'].dropna().iloc[i])

[{'id': 'ENSR00000222749', 'type': 'promoter_flanking_region', 'consequence': ['regulatory_region_variant']}]
[{'id': 'ENSR00000330098', 'type': 'CTCF_binding_site', 'consequence': ['regulatory_region_variant']}]
[{'id': 'ENSR00000330098', 'type': 'CTCF_binding_site', 'consequence': ['regulatory_region_variant']}]
[{'id': 'ENSR00000330098', 'type': 'CTCF_binding_site', 'consequence': ['regulatory_region_variant']}]
[{'id': 'ENSR00000330099', 'type': 'CTCF_binding_site', 'consequence': ['regulatory_region_variant']}]


In [22]:
for i in range(len(final_df["transcripts"])):
    print(f'Variant {final_df["vid"].iloc[i]} has {len(final_df["transcripts"].iloc[i])} affected transcripts')

Variant 8-161062-C-T has 1 affected transcripts
Variant 8-161167-C-T has 2 affected transcripts
Variant 8-161176-T-C has 2 affected transcripts
Variant 8-161240-A-G has 2 affected transcripts
Variant 8-163226-T-C has 2 affected transcripts
Variant 8-163249-T-C has 2 affected transcripts
Variant 8-163366-T-C has 2 affected transcripts
Variant 8-163370-C-G has 2 affected transcripts
Variant 8-163387-C-T has 2 affected transcripts
Variant 8-163432-A-G has 2 affected transcripts
Variant 8-163438-C-T has 2 affected transcripts
Variant 8-163550-A-G has 2 affected transcripts
Variant 8-163553-T-C has 2 affected transcripts
Variant 8-163654-C-T has 2 affected transcripts
Variant 8-163784-C-G has 2 affected transcripts
Variant 8-169283-T-G has 2 affected transcripts
Variant 8-169305-G-C has 2 affected transcripts
Variant 8-169366-T-C has 2 affected transcripts
Variant 8-169403-G-A has 2 affected transcripts
Variant 8-169422-G-C has 2 affected transcripts
Variant 8-169428-A-C has 2 affected tran

In [23]:
def process_transcripts(final_df):
    pass

Index(['chromosome', 'position', 'refAllele', 'altAlleles', 'quality',
       'filters', 'fisherStrandBias', 'mappingQuality', 'cytogeneticBand',
       'vid', 'chromosome', 'begin', 'end', 'refAllele', 'altAllele',
       'variantType', 'hgvsg', 'phylopScore', 'dannScore', 'transcripts',
       'dbsnp', 'gnomad', 'topmed', 'gerpScore', 'regulatoryRegions',
       'inLowComplexityRegion'],
      dtype='object')